In [159]:
import requests
import re
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from datetime import datetime

In [160]:
def get_url(title, location):
    template_url = 'https://www.indeed.com/jobs?q={}&l={}&fromage=14'
    url = template_url.format(urllib.parse.quote(title), urllib.parse.quote(location))
    return url

In [161]:
get_url('Data scientist', 'United states')

'https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=14'

In [162]:
def get_records(card):

    try:
        job_title = card.find('h2', {'class':'jobTitle'}).findAll('span')[-1].text
    except:
        job_title = ''
    
    try:
        job_conpany = card.find('span', {'class':'companyName'}).text
    except:
        job_conpany = ''

    try:
        job_rating = card.find('span', {'class':'ratingsDisplay withRatingLink'}).text
    except:
        job_rating = ''
    
    try:
        job_location = [i.text for i in card.find_all('div', {'class':'companyLocation'})]
    except:
        job_location = ''
    
    try:
        job_salary =  card.find('span', {'class':'salary-snippet'}).text
    except:
        job_salary = ''
    
    try:
        job_date = card.find('span', {'class':'date'}).text
    except:
        job_date = ''

    extract_date = datetime.today().strftime('%Y-%m-%d')

    job_url = 'https://www.indeed.com'+ card['href']
    return job_title, job_conpany, job_rating, job_location, job_salary, job_date, extract_date, job_url


In [163]:
def get_detail(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    #job detail
    info = {}
    try:
        job_detail = soup.find_all('div', {'class':'jobsearch-JobDescriptionSection-sectionItem'})
        for detail in job_detail:
            if detail.find('span'):
                info[detail.find_all('div')[0].text] = [i.text for i in detail.find_all('span')]
            else:
                info[detail.find_all('div')[0].text] = [i.text for i in detail.find_all('div')[1:]]
    except:
        pass
    
    #job qualification
    try:
        jobQual = soup.find('div', {'id':'qualificationsSection'})
        title = jobQual.find('h2').text
        info[title] = [i.text for i in jobQual.find_all('li', {'class':'icl-u-xs-p--none jobsearch-ReqAndQualSection-item'})]
    except:
        pass

    #job description
    try:
        jobdesc = soup.find('div', {'id':'jobDescriptionText'})
        info['Description'] = jobdesc.format_string
    except:
        pass
        
    res.close()
    return info



In [164]:
def get_info(title, location):
    records = []
    details = []
    no_of_jobs = {}
    url = get_url(title, location)
    for i in range(1):
        if i > 35:
            wait = randint(30, 50)
            sleep(wait)
        print(url)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        #get total jobs
        no_of_job_string = soup.find(id='searchCountPages').text.strip()
        no_of_job = int(re.sub('\W+', '', no_of_job_string.split()[3]))
        no_of_jobs[title] = no_of_job
        try:
            cards = soup.find(id='mosaic-provider-jobcards').findChildren("a" , recursive=False)
        except:
            continue

        for card in cards:
            record = get_records(card)
            records.append(record)
            wait = randint(5, 12)
            sleep(wait)
            detail = get_detail(record[-1])
            details.append(detail)
        
        try:
            url = 'https://www.indeed.com'+soup.find('a', {'aria-label':"Next"})['href']
            wait = randint(1, 10)
            sleep(wait)
        except:
            break
        res.close()
    return records, details, no_of_jobs

In [165]:
def saveDate(title, records, details):
    col = ['Title', 'Company', 'Rating', 'Location', 'Salary', 'Post_date', 'Extract_date', 'Job_url']
    df_basic = pd.DataFrame(records)
    df_basic.columns = col
    df_detail = pd.DataFrame.from_dict(details)
    df = pd.concat([df_basic, df_detail], axis=1)
    df.to_csv(f'{title}.csv')

In [166]:
no_of_jobs = {}

#get records from 'Data scientist' in 'UniedStates'
records, details, total = get_info('Data scientist', 'United states')
saveDate('Data scientist', records, details)
no_of_jobs['Data scientist'] = total

sleep(300)
#get records from 'Software programmer' in 'UniedStates'
records, details, total = get_info('Software programmer', 'United states')
saveDate('Software programmer', records, details)
no_of_jobs['Software programmer'] = total

sleep(300)
#get records from 'Web programmer' in 'UniedStates'
records, details, total = get_info('Web programme', 'United states')
saveDate('Web programmer', records, details)
no_of_jobs['Web programmer'] = total

sleep(300)
#get records from 'Information architect' in 'UniedStates'
records, details, total = get_info('Information architect', 'United states')
saveDate('Information architect', records, details)
no_of_jobs['Information architect'] = total

https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=14


AttributeError: 'NoneType' object has no attribute 'text'